In [1]:
import re
import pickle
import warnings
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from xgboost import XGBRegressor
from datetime import date, datetime
from sklearn.impute import SimpleImputer
from IPython.core.display import display, HTML
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
warnings.filterwarnings('ignore')
pd.pandas.set_option('display.max_rows', None)
pd.pandas.set_option('display.max_columns', None)
display(HTML('<style>.container{width : 100% ! important;}</style>'))

In [2]:
df = pd.read_csv('IPL.csv')
df.head()

,id,date,city,venue,1st_team,neutral_venue,2nd_team,toss_winner,toss_decision,innings,batting_team,bowling_team,batsman,non_striker,bowler,overs,balls,ball_no,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,runs,wickets,runs_last_5_overs,wickets_last_5_overs,balls_remaining,runs_scored_ball_remaining,winner,result,result_margin,player_of_match,eliminator,method,umpire1,umpire2,final_score,total_wickets,total_balls
0,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,1,0,1,1,0,0,NaN,NaN,NaN,legbyes,1,0,1,0,119,221,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,120
1,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,2,2,0,0,0,0,0,NaN,NaN,NaN,NaN,1,0,1,0,118,221,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,120
2,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,3,3,0,1,1,0,0,NaN,NaN,NaN,wides,2,0,2,0,117,220,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,120
3,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,4,3,0,0,0,0,0,NaN,NaN,NaN,NaN,2,0,2,0,117,220,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,120
4,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,5,4,0,0,0,0,0,NaN,NaN,NaN,NaN,2,0,2,0,116,220,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,120


In [3]:
df.columns

Index(['id', 'date', 'city', 'venue', '1st_team', 'neutral_venue', '2nd_team',
       'toss_winner', 'toss_decision', 'innings', 'batting_team',
       'bowling_team', 'batsman', 'non_striker', 'bowler', 'overs', 'balls',
       'ball_no', 'batsman_runs', 'extra_runs', 'total_runs', 'non_boundary',
       'is_wicket', 'dismissal_kind', 'player_dismissed', 'fielder',
       'extras_type', 'runs', 'wickets', 'runs_last_5_overs',
       'wickets_last_5_overs', 'balls_remaining', 'runs_scored_ball_remaining',
       'winner', 'result', 'result_margin', 'player_of_match', 'eliminator',
       'method', 'umpire1', 'umpire2', 'final_score', 'total_wickets',
       'total_balls'],
      dtype='object')

In [4]:
df = df[['date', 'venue', 'toss_decision', 'innings', 'batting_team', 'bowling_team', 'overs', 'balls', 'ball_no', 'runs', 'wickets', 
         'runs_last_5_overs', 'wickets_last_5_overs', 'balls_remaining', 'runs_scored_ball_remaining', 'final_score']]

In [5]:
df.head()

,date,venue,toss_decision,innings,batting_team,bowling_team,overs,balls,ball_no,runs,wickets,runs_last_5_overs,wickets_last_5_overs,balls_remaining,runs_scored_ball_remaining,final_score
0,18-04-2008,M Chinnaswamy Stadium,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,1,1,0,1,0,119,221,222
1,18-04-2008,M Chinnaswamy Stadium,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,2,1,0,1,0,118,221,222
2,18-04-2008,M Chinnaswamy Stadium,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,3,2,0,2,0,117,220,222
3,18-04-2008,M Chinnaswamy Stadium,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,3,2,0,2,0,117,220,222
4,18-04-2008,M Chinnaswamy Stadium,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,4,2,0,2,0,116,220,222


In [6]:
df['year'] = df['date'].apply(lambda dateString : datetime.strptime(dateString,'%d-%m-%Y').year)
df.drop('date', axis = 1, inplace = True)

In [7]:
df.head()

,venue,toss_decision,innings,batting_team,bowling_team,overs,balls,ball_no,runs,wickets,runs_last_5_overs,wickets_last_5_overs,balls_remaining,runs_scored_ball_remaining,final_score,year
0,M Chinnaswamy Stadium,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,1,1,0,1,0,119,221,222,2008
1,M Chinnaswamy Stadium,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,2,1,0,1,0,118,221,222,2008
2,M Chinnaswamy Stadium,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,3,2,0,2,0,117,220,222,2008
3,M Chinnaswamy Stadium,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,3,2,0,2,0,117,220,222,2008
4,M Chinnaswamy Stadium,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,4,2,0,2,0,116,220,222,2008


In [8]:
df.head()

,venue,toss_decision,innings,batting_team,bowling_team,overs,balls,ball_no,runs,wickets,runs_last_5_overs,wickets_last_5_overs,balls_remaining,runs_scored_ball_remaining,final_score,year
0,M Chinnaswamy Stadium,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,1,1,0,1,0,119,221,222,2008
1,M Chinnaswamy Stadium,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,2,1,0,1,0,118,221,222,2008
2,M Chinnaswamy Stadium,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,3,2,0,2,0,117,220,222,2008
3,M Chinnaswamy Stadium,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,3,2,0,2,0,117,220,222,2008
4,M Chinnaswamy Stadium,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,4,2,0,2,0,116,220,222,2008


In [9]:
df.columns

Index(['venue', 'toss_decision', 'innings', 'batting_team', 'bowling_team',
       'overs', 'balls', 'ball_no', 'runs', 'wickets', 'runs_last_5_overs',
       'wickets_last_5_overs', 'balls_remaining', 'runs_scored_ball_remaining',
       'final_score', 'year'],
      dtype='object')

In [10]:
dummy = ['venue', 'toss_decision', 'batting_team', 'bowling_team']
df_dummy = pd.get_dummies(df[dummy], prefix_sep = '_', drop_first = False)
df = df.drop(dummy, axis = 1)
df = pd.concat([df, df_dummy], axis = 1)

In [11]:
df.head()

,innings,overs,balls,ball_no,runs,wickets,runs_last_5_overs,wickets_last_5_overs,balls_remaining,runs_scored_ball_remaining,final_score,year,venue_Barabati Stadium,venue_Brabourne Stadium,venue_Buffalo Park,venue_De Beers Diamond Oval,venue_Dr DY Patil Sports Academy,venue_Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium,venue_Dubai International Cricket Stadium,venue_Eden Gardens,venue_Feroz Shah Kotla,venue_Green Park,venue_Himachal Pradesh Cricket Association Stadium,venue_Holkar Cricket Stadium,venue_JSCA International Stadium Complex,venue_Kingsmead,venue_M Chinnaswamy Stadium,"venue_MA Chidambaram Stadium, Chepauk","venue_Narendra Modi Stadium, Ahmedabad",venue_Nehru Stadium,venue_New Wanderers Stadium,venue_Newlands,venue_OUTsurance Oval,"venue_Punjab Cricket Association Stadium, Mohali","venue_Rajiv Gandhi International Stadium, Uppal",venue_Saurashtra Cricket Association Stadium,venue_Sawai Mansingh Stadium,venue_Shaheed Veer Narayan Singh International Stadium,venue_Sharjah Cricket Stadium,venue_Sheikh Zayed Stadium,venue_St George's Park,venue_Subrata Roy Sahara Stadium,venue_SuperSport Park,"venue_Vidarbha Cricket Association Stadium, Jamtha",venue_Wankhede Stadium,toss_decision_bat,toss_decision_field,batting_team_Chennai Super Kings,batting_team_Deccan Chargers,batting_team_Delhi Capitals,batting_team_Gujarat Lions,batting_team_Kochi Tuskers Kerala,batting_team_Kolkata Knight Riders,batting_team_Mumbai Indians,batting_team_Pune Warriors,batting_team_Punjab Kings,batting_team_Rajasthan Royals,batting_team_Rising Pune Supergiant,batting_team_Royal Challengers Bangalore,batting_team_Sunrisers Hyderabad,bowling_team_Chennai Super Kings,bowling_team_Deccan Chargers,bowling_team_Delhi Capitals,bowling_team_Gujarat Lions,bowling_team_Kochi Tuskers Kerala,bowling_team_Kolkata Knight Riders,bowling_team_Mumbai Indians,bowling_team_Pune Warriors,bowling_team_Punjab Kings,bowling_team_Rajasthan Royals,bowling_team_Rising Pune Supergiant,bowling_team_Royal Challengers Bangalore,bowling_team_Sunrisers Hyderabad
0,1,0,1,1,1,0,1,0,119,221,222,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,0,2,2,1,0,1,0,118,221,222,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1,0,3,3,2,0,2,0,117,220,222,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1,0,4,3,2,0,2,0,117,220,222,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1,0,5,4,2,0,2,0,116,220,222,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [12]:
df.shape

(198569, 73)

In [13]:
df.columns

Index(['innings', 'overs', 'balls', 'ball_no', 'runs', 'wickets',
       'runs_last_5_overs', 'wickets_last_5_overs', 'balls_remaining',
       'runs_scored_ball_remaining', 'final_score', 'year',
       'venue_Barabati Stadium', 'venue_Brabourne Stadium',
       'venue_Buffalo Park', 'venue_De Beers Diamond Oval',
       'venue_Dr DY Patil Sports Academy',
       'venue_Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'venue_Dubai International Cricket Stadium', 'venue_Eden Gardens',
       'venue_Feroz Shah Kotla', 'venue_Green Park',
       'venue_Himachal Pradesh Cricket Association Stadium',
       'venue_Holkar Cricket Stadium',
       'venue_JSCA International Stadium Complex', 'venue_Kingsmead',
       'venue_M Chinnaswamy Stadium', 'venue_MA Chidambaram Stadium, Chepauk',
       'venue_Narendra Modi Stadium, Ahmedabad', 'venue_Nehru Stadium',
       'venue_New Wanderers Stadium', 'venue_Newlands',
       'venue_OUTsurance Oval',
       'venue_Punjab Cricket Associa

In [14]:
columns = []
for i in df.columns:
    i = re.sub('\s', '_', i)
    i = re.sub('\.', '', i)
    i = re.sub('\,', '', i)
    i = re.sub('\-', '_', i)
    columns.append(i)
df.columns = columns

In [15]:
df.columns

Index(['innings', 'overs', 'balls', 'ball_no', 'runs', 'wickets',
       'runs_last_5_overs', 'wickets_last_5_overs', 'balls_remaining',
       'runs_scored_ball_remaining', 'final_score', 'year',
       'venue_Barabati_Stadium', 'venue_Brabourne_Stadium',
       'venue_Buffalo_Park', 'venue_De_Beers_Diamond_Oval',
       'venue_Dr_DY_Patil_Sports_Academy',
       'venue_Dr_YS_Rajasekhara_Reddy_ACA_VDCA_Cricket_Stadium',
       'venue_Dubai_International_Cricket_Stadium', 'venue_Eden_Gardens',
       'venue_Feroz_Shah_Kotla', 'venue_Green_Park',
       'venue_Himachal_Pradesh_Cricket_Association_Stadium',
       'venue_Holkar_Cricket_Stadium',
       'venue_JSCA_International_Stadium_Complex', 'venue_Kingsmead',
       'venue_M_Chinnaswamy_Stadium', 'venue_MA_Chidambaram_Stadium_Chepauk',
       'venue_Narendra_Modi_Stadium_Ahmedabad', 'venue_Nehru_Stadium',
       'venue_New_Wanderers_Stadium', 'venue_Newlands',
       'venue_OUTsurance_Oval',
       'venue_Punjab_Cricket_Association_

In [16]:
df.head()

,innings,overs,balls,ball_no,runs,wickets,runs_last_5_overs,wickets_last_5_overs,balls_remaining,runs_scored_ball_remaining,final_score,year,venue_Barabati_Stadium,venue_Brabourne_Stadium,venue_Buffalo_Park,venue_De_Beers_Diamond_Oval,venue_Dr_DY_Patil_Sports_Academy,venue_Dr_YS_Rajasekhara_Reddy_ACA_VDCA_Cricket_Stadium,venue_Dubai_International_Cricket_Stadium,venue_Eden_Gardens,venue_Feroz_Shah_Kotla,venue_Green_Park,venue_Himachal_Pradesh_Cricket_Association_Stadium,venue_Holkar_Cricket_Stadium,venue_JSCA_International_Stadium_Complex,venue_Kingsmead,venue_M_Chinnaswamy_Stadium,venue_MA_Chidambaram_Stadium_Chepauk,venue_Narendra_Modi_Stadium_Ahmedabad,venue_Nehru_Stadium,venue_New_Wanderers_Stadium,venue_Newlands,venue_OUTsurance_Oval,venue_Punjab_Cricket_Association_Stadium_Mohali,venue_Rajiv_Gandhi_International_Stadium_Uppal,venue_Saurashtra_Cricket_Association_Stadium,venue_Sawai_Mansingh_Stadium,venue_Shaheed_Veer_Narayan_Singh_International_Stadium,venue_Sharjah_Cricket_Stadium,venue_Sheikh_Zayed_Stadium,venue_St_George's_Park,venue_Subrata_Roy_Sahara_Stadium,venue_SuperSport_Park,venue_Vidarbha_Cricket_Association_Stadium_Jamtha,venue_Wankhede_Stadium,toss_decision_bat,toss_decision_field,batting_team_Chennai_Super_Kings,batting_team_Deccan_Chargers,batting_team_Delhi_Capitals,batting_team_Gujarat_Lions,batting_team_Kochi_Tuskers_Kerala,batting_team_Kolkata_Knight_Riders,batting_team_Mumbai_Indians,batting_team_Pune_Warriors,batting_team_Punjab_Kings,batting_team_Rajasthan_Royals,batting_team_Rising_Pune_Supergiant,batting_team_Royal_Challengers_Bangalore,batting_team_Sunrisers_Hyderabad,bowling_team_Chennai_Super_Kings,bowling_team_Deccan_Chargers,bowling_team_Delhi_Capitals,bowling_team_Gujarat_Lions,bowling_team_Kochi_Tuskers_Kerala,bowling_team_Kolkata_Knight_Riders,bowling_team_Mumbai_Indians,bowling_team_Pune_Warriors,bowling_team_Punjab_Kings,bowling_team_Rajasthan_Royals,bowling_team_Rising_Pune_Supergiant,bowling_team_Royal_Challengers_Bangalore,bowling_team_Sunrisers_Hyderabad
0,1,0,1,1,1,0,1,0,119,221,222,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,0,2,2,1,0,1,0,118,221,222,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1,0,3,3,2,0,2,0,117,220,222,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1,0,4,3,2,0,2,0,117,220,222,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1,0,5,4,2,0,2,0,116,220,222,2008,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [17]:
df = df[['year', 'batting_team_Chennai_Super_Kings', 'batting_team_Deccan_Chargers', 'batting_team_Delhi_Capitals', 
         'batting_team_Gujarat_Lions', 'batting_team_Kochi_Tuskers_Kerala', 'batting_team_Kolkata_Knight_Riders', 
         'batting_team_Mumbai_Indians', 'batting_team_Pune_Warriors', 'batting_team_Punjab_Kings', 'batting_team_Rajasthan_Royals', 
         'batting_team_Rising_Pune_Supergiant', 'batting_team_Royal_Challengers_Bangalore', 'batting_team_Sunrisers_Hyderabad', 
         'bowling_team_Chennai_Super_Kings', 'bowling_team_Deccan_Chargers', 'bowling_team_Delhi_Capitals', 'bowling_team_Gujarat_Lions', 
         'bowling_team_Kochi_Tuskers_Kerala', 'bowling_team_Kolkata_Knight_Riders', 'bowling_team_Mumbai_Indians', 
         'bowling_team_Pune_Warriors', 'bowling_team_Punjab_Kings', 'bowling_team_Rajasthan_Royals', 'bowling_team_Rising_Pune_Supergiant', 
         'bowling_team_Royal_Challengers_Bangalore', 'bowling_team_Sunrisers_Hyderabad', 'toss_decision_bat', 'toss_decision_field', 
         'venue_Barabati_Stadium', 'venue_Brabourne_Stadium', 'venue_Buffalo_Park', 'venue_De_Beers_Diamond_Oval', 
         'venue_Dr_DY_Patil_Sports_Academy', 'venue_Dr_YS_Rajasekhara_Reddy_ACA_VDCA_Cricket_Stadium', 
         'venue_Dubai_International_Cricket_Stadium', 'venue_Eden_Gardens', 'venue_Feroz_Shah_Kotla', 'venue_Green_Park', 
         'venue_Himachal_Pradesh_Cricket_Association_Stadium', 'venue_Holkar_Cricket_Stadium', 'venue_JSCA_International_Stadium_Complex', 
         'venue_Kingsmead', 'venue_M_Chinnaswamy_Stadium', 'venue_MA_Chidambaram_Stadium_Chepauk', 'venue_Narendra_Modi_Stadium_Ahmedabad', 
         'venue_Nehru_Stadium', 'venue_New_Wanderers_Stadium', 'venue_Newlands', 'venue_OUTsurance_Oval', 
         'venue_Punjab_Cricket_Association_Stadium_Mohali', 'venue_Rajiv_Gandhi_International_Stadium_Uppal', 
         'venue_Saurashtra_Cricket_Association_Stadium', 'venue_Sawai_Mansingh_Stadium', 
         'venue_Shaheed_Veer_Narayan_Singh_International_Stadium', 'venue_Sharjah_Cricket_Stadium', 'venue_Sheikh_Zayed_Stadium', 
         'venue_St_George\'s_Park', 'venue_Subrata_Roy_Sahara_Stadium', 'venue_SuperSport_Park', 
         'venue_Vidarbha_Cricket_Association_Stadium_Jamtha', 'venue_Wankhede_Stadium', 'innings', 'overs', 'balls', 'ball_no', 'runs', 
         'wickets', 'runs_last_5_overs', 'wickets_last_5_overs', 'balls_remaining', 'runs_scored_ball_remaining', 'final_score']]

In [18]:
train = df[df['year'] <= 2018]
test = df[df['year'] > 2018]

In [19]:
X_train = pd.DataFrame(train.drop(['year', 'final_score'], axis = 1), columns = train.drop(['year', 'final_score'], axis = 1).columns)
X_test = pd.DataFrame(test.drop(['year', 'final_score'], axis = 1), columns = test.drop(['year', 'final_score'], axis = 1).columns)
y_train = train['final_score']
y_test = test['final_score']

In [20]:
X_train.columns

Index(['batting_team_Chennai_Super_Kings', 'batting_team_Deccan_Chargers',
       'batting_team_Delhi_Capitals', 'batting_team_Gujarat_Lions',
       'batting_team_Kochi_Tuskers_Kerala',
       'batting_team_Kolkata_Knight_Riders', 'batting_team_Mumbai_Indians',
       'batting_team_Pune_Warriors', 'batting_team_Punjab_Kings',
       'batting_team_Rajasthan_Royals', 'batting_team_Rising_Pune_Supergiant',
       'batting_team_Royal_Challengers_Bangalore',
       'batting_team_Sunrisers_Hyderabad', 'bowling_team_Chennai_Super_Kings',
       'bowling_team_Deccan_Chargers', 'bowling_team_Delhi_Capitals',
       'bowling_team_Gujarat_Lions', 'bowling_team_Kochi_Tuskers_Kerala',
       'bowling_team_Kolkata_Knight_Riders', 'bowling_team_Mumbai_Indians',
       'bowling_team_Pune_Warriors', 'bowling_team_Punjab_Kings',
       'bowling_team_Rajasthan_Royals', 'bowling_team_Rising_Pune_Supergiant',
       'bowling_team_Royal_Challengers_Bangalore',
       'bowling_team_Sunrisers_Hyderabad', 'to

In [21]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((164665, 71), (33904, 71), (164665,), (33904,))

In [22]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [23]:
y_train_pred_lr = lr.predict(X_train)
y_test_pred_lr = lr.predict(X_test)
print('Linear Regression:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_lr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_lr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_lr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_lr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_lr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_lr) / y_test))
error_lr = (np.mean(mean_absolute_error(y_test, y_test_pred_lr) / y_test) / 2)

Linear Regression:
----------------------------------------------------------------------------------------------------
Train r2_score:  1.0
Test r2_score:  1.0
----------------------------------------------------------------------------------------------------
Train RMSE:  1.0107948596612043e-12
Test RMSE:  9.296898652776329e-13
----------------------------------------------------------------------------------------------------
Train MAPE:  5.448935465169234e-15
Test MAPE:  4.880906804192122e-15


In [24]:
data_lr = pd.DataFrame()
data_lr['actual'] = y_test
data_lr['predicted'] = y_test_pred_lr
data_lr['lower range'] = y_test_pred_lr - (error_lr * y_test_pred_lr)
data_lr['upper range'] = y_test_pred_lr + (error_lr * y_test_pred_lr)
t_list = []
for i in data_lr.index:
    if data_lr['actual'][i] > data_lr['lower range'][i]:
        if data_lr['actual'][i] < data_lr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_lr['Bool'] = t_list
data_lr.to_csv('data_lr.csv', index = None)
data_lr['Bool'].value_counts()/data_lr.shape[0]

False    0.711656
True     0.288344
Name: Bool, dtype: float64

In [25]:
la = Lasso()
possible_parameter_values = {'alpha' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
la_rscv = RandomizedSearchCV(estimator = la, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
la_rscv.fit(X_train, y_train)
la_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] END ........................................alpha=0.624; total time=   0.9s
[CV] END ........................................alpha=0.624; total time=   0.9s
[CV] END ........................................alpha=0.624; total time=   1.1s
[CV] END ........................................alpha=0.624; total time=   1.1s
[CV] END ........................................alpha=0.624; total time=   0.8s
[CV] END ........................................alpha=0.624; total time=   1.2s
[CV] END ........................................alpha=0.624; total time=   1.2s
[CV] END ........................................alpha=0.624; total time=   1.1s
[CV] END ........................................alpha=0.624; total time=   0.9s
[CV] END ........................................alpha=0.624; total time=   1.0s
[CV] END ........................................alpha=0.242; total time=   1.2s
[CV] END .....................................

{'alpha': 0.032}

In [26]:
y_train_pred_la = la_rscv.predict(X_train)
y_test_pred_la = la_rscv.predict(X_test)
print('Lasso Regression:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_la))
print('Test r2_score: ', r2_score(y_test, y_test_pred_la))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_la)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_la)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_la) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_la) / y_test))
error_la = (np.mean(mean_absolute_error(y_test, y_test_pred_la) / y_test) / 2)

Lasso Regression:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.999999990518285
Test r2_score:  0.9999999904387121
----------------------------------------------------------------------------------------------------
Train RMSE:  0.002882507913428022
Test RMSE:  0.0028717110674475305
----------------------------------------------------------------------------------------------------
Train MAPE:  1.4828694196380799e-05
Test MAPE:  1.4431001674513433e-05


In [27]:
data_la = pd.DataFrame()
data_la['actual'] = y_test
data_la['predicted'] = y_test_pred_la
data_la['lower range'] = y_test_pred_la - (error_la * y_test_pred_la)
data_la['upper range'] = y_test_pred_la + (error_la * y_test_pred_la)
t_list = []
for i in data_la.index:
    if data_la['actual'][i] > data_la['lower range'][i]:
        if data_la['actual'][i] < data_la['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_la['Bool'] = t_list
data_la.to_csv('data_la.csv', index = None)
data_la['Bool'].value_counts()/data_la.shape[0]

False    0.678445
True     0.321555
Name: Bool, dtype: float64

In [28]:
ri = Ridge()
possible_parameter_values = {'alpha' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
ri_rscv = RandomizedSearchCV(estimator = ri, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
ri_rscv.fit(X_train, y_train)
ri_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] END ........................................alpha=0.624; total time=   0.1s
[CV] END ........................................alpha=0.624; total time=   0.1s
[CV] END ........................................alpha=0.624; total time=   0.2s
[CV] END ........................................alpha=0.624; total time=   0.1s
[CV] END ........................................alpha=0.624; total time=   0.1s
[CV] END ........................................alpha=0.624; total time=   0.2s
[CV] END ........................................alpha=0.624; total time=   0.1s
[CV] END ........................................alpha=0.624; total time=   0.1s
[CV] END ........................................alpha=0.624; total time=   0.1s
[CV] END ........................................alpha=0.624; total time=   0.2s
[CV] END ........................................alpha=0.242; total time=   0.2s
[CV] END .....................................

{'alpha': 0.032}

In [29]:
y_train_pred_ri = ri_rscv.predict(X_train)
y_test_pred_ri = ri_rscv.predict(X_test)
print('Ridge Regression:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_ri))
print('Test r2_score: ', r2_score(y_test, y_test_pred_ri))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_ri)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_ri)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_ri) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_ri) / y_test))
error_ri = (np.mean(mean_absolute_error(y_test, y_test_pred_ri) / y_test) / 2)

Ridge Regression:
----------------------------------------------------------------------------------------------------
Train r2_score:  1.0
Test r2_score:  1.0
----------------------------------------------------------------------------------------------------
Train RMSE:  1.9473897914650207e-08
Test RMSE:  2.188653248146828e-08
----------------------------------------------------------------------------------------------------
Train MAPE:  1.0317972775017154e-10
Test MAPE:  1.0963735640703638e-10


In [30]:
data_ri = pd.DataFrame()
data_ri['actual'] = y_test
data_ri['predicted'] = y_test_pred_ri
data_ri['lower range'] = y_test_pred_ri - (error_ri * y_test_pred_ri)
data_ri['upper range'] = y_test_pred_ri + (error_ri * y_test_pred_ri)
t_list = []
for i in data_ri.index:
    if data_ri['actual'][i] > data_ri['lower range'][i]:
        if data_ri['actual'][i] < data_ri['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_ri['Bool'] = t_list
data_ri.to_csv('data_ri.csv', index = None)
data_ri['Bool'].value_counts()/data_ri.shape[0]

False    0.67523
True     0.32477
Name: Bool, dtype: float64

In [31]:
# dtr = DecisionTreeRegressor()
# possible_parameter_values = {'criterion' : ['mse', 'friedman_mse', 'mae'],
#                             'splitter' : ['best', 'random'],
#                             'max_depth' : [int(x) for x in np.arange(start = 5, stop = 101, step = 5)],
#                             'min_samples_split' : [int(x) for x in np.arange(start = 5, stop = 101, step = 1)],
#                             'min_samples_leaf' : [int(x) for x in np.arange(start = 1, stop = 51, step = 1)]}
# dtr_rscv = RandomizedSearchCV(estimator = dtr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
#                              n_jobs = 1)
# dtr_rscv.fit(X_train, y_train)
# dtr_rscv.best_params_

In [32]:
#y_train_pred_dtr = dtr_rscv.predict(X_train)
#y_test_pred_dtr = dtr_rscv.predict(X_test)
#print('Decision Tree Regressor:')
#print('-' * 100)
#print('Train r2_score: ', r2_score(y_train, y_train_pred_dtr))
#print('Test r2_score: ', r2_score(y_test, y_test_pred_dtr))
#print('-' * 100)
#print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_dtr)))
#print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_dtr)))
#print('-' * 100)
#print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_dtr) / y_train))
#print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_dtr) / y_test))
#error_dtr = np.mean(mean_absolute_error(y_test, y_test_pred_dtr) / y_test)

In [33]:
#data_dtr = pd.DataFrame()
#data_dtr['actual'] = y_test
#data_dtr['predicted'] = y_test_pred_dtr
#data_dtr['lower range'] = y_test_pred_dtr - (error_dtr * y_test_pred_dtr)
#data_dtr['upper range'] = y_test_pred_dtr + (error_dtr * y_test_pred_dtr)
#t_list = []
#for i in data_dtr.index:
#    if data_dtr['actual'][i] > data_dtr['lower range'][i]:
#        if data_dtr['actual'][i] < data_dtr['upper range'][i]:
#            t_list.append('True')
#        else:
#            t_list.append('False')
#    else:
#        t_list.append('False')
#data_dtr['Bool'] = t_list
#data_dtr['Bool'].value_counts()/data_dtr.shape[0]

In [35]:
# rfr = RandomForestRegressor()
# possible_parameter_values = {'n_estimators' : [int(x) for x in np.arange(start = 50, stop = 1001, step = 50)],
#                             'max_depth' : [int(x) for x in np.arange(start = 5, stop = 101, step = 5)],
#                             'min_samples_split' : [int(x) for x in np.arange(start = 5, stop = 101, step = 1)],
#                             'min_samples_leaf' : [int(x) for x in np.arange(start = 1, stop = 51, step = 1)]}
# rfr_rscv = RandomizedSearchCV(estimator = rfr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
#                              n_jobs = 1)
# rfr_rscv.fit(X_train, y_train)
# rfr_rscv.best_params_

In [36]:
#y_train_pred_rfr = rfr_rscv.predict(X_train)
#y_test_pred_rfr = rfr_rscv.predict(X_test)
#print('Random Forest Regressor:')
#print('-' * 100)
#print('Train r2_score: ', r2_score(y_train, y_train_pred_rfr))
#print('Test r2_score: ', r2_score(y_test, y_test_pred_rfr))
#print('-' * 100)
#print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_rfr)))
#print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_rfr)))
#print('-' * 100)
#print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_rfr) / y_train))
#print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_rfr) / y_test))
#error_rfr = np.mean(mean_absolute_error(y_test, y_test_pred_rfr) / y_test)

In [37]:
#data_rfr = pd.DataFrame()
#data_rfr['actual'] = y_test
#data_rfr['predicted'] = y_test_pred_rfr
#data_rfr['lower range'] = y_test_pred_rfr - (error_rfr * y_test_pred_rfr)
#data_rfr['upper range'] = y_test_pred_rfr + (error_rfr * y_test_pred_rfr)
#t_list = []
#for i in data_rfr.index:
#    if data_rfr['actual'][i] > data_rfr['lower range'][i]:
#        if data_rfr['actual'][i] < data_rfr['upper range'][i]:
#            t_list.append('True')
#        else:
#            t_list.append('False')
#    else:
#        t_list.append('False')
#data_rfr['Bool'] = t_list
#data_rfr['Bool'].value_counts()/data_rfr.shape[0]

In [39]:
# abr = AdaBoostRegressor()
# possible_parameter_values = {'n_estimators' : [int(x) for x in np.arange(start = 50, stop = 1001, step = 50)],
#                              'learning_rate' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
# abr_rscv = RandomizedSearchCV(estimator = abr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
#                               n_jobs = 1)
# abr_rscv.fit(X_train, y_train)
# abr_rscv.best_params_

In [40]:
# y_train_pred_abr = abr_rscv.predict(X_train)
# y_test_pred_abr = abr_rscv.predict(X_test)
# print('Ada Boost Regressor:')
# print('-' * 100)
# print('Train r2_score: ', r2_score(y_train, y_train_pred_abr))
# print('Test r2_score: ', r2_score(y_test, y_test_pred_abr))
# print('-' * 100)
# print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_abr)))
# print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_abr)))
# print('-' * 100)
# print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_abr) / y_train))
# print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_abr) / y_test))
# error_abr = (np.mean(mean_absolute_error(y_test, y_test_pred_abr) / y_test) / 2)

In [41]:
# data_abr = pd.DataFrame()
# data_abr['actual'] = y_test
# data_abr['predicted'] = y_test_pred_abr
# data_abr['lower range'] = y_test_pred_abr - (error_abr * y_test_pred_abr)
# data_abr['upper range'] = y_test_pred_abr + (error_abr * y_test_pred_abr)
# t_list = []
# for i in data_abr.index:
#     if data_abr['actual'][i] > data_abr['lower range'][i]:
#         if data_abr['actual'][i] < data_abr['upper range'][i]:
#             t_list.append('True')
#         else:
#             t_list.append('False')
#     else:
#         t_list.append('False')
# data_abr['Bool'] = t_list
# data_abr['Bool'].value_counts()/data_abr.shape[0]

In [43]:
# knr = KNeighborsRegressor()
# possible_parameter_values = {'n_neighbors' : [int(x) for x in np.arange(start = 1, stop = 11, step = 1)],
#                              'weights' : ['uniform', 'distance']}
# knr_rscv = RandomizedSearchCV(estimator = knr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
#                               n_jobs = 1)
# knr_rscv.fit(X_train, y_train)
# knr_rscv.best_params_

In [44]:
# y_train_pred_knr = knr_rscv.predict(X_train)
# y_test_pred_knr = knr_rscv.predict(X_test)
# print('K Neighbors Regressor:')
# print('-' * 100)
# print('Train r2_score: ', r2_score(y_train, y_train_pred_knr))
# print('Test r2_score: ', r2_score(y_test, y_test_pred_knr))
# print('-' * 100)
# print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_knr)))
# print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_knr)))
# print('-' * 100)
# print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_knr) / y_train))
# print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_knr) / y_test))
# error_knr = (np.mean(mean_absolute_error(y_test, y_test_pred_knr) / y_test) / 2)

In [45]:
# data_knr = pd.DataFrame()
# data_knr['actual'] = y_test
# data_knr['predicted'] = y_test_pred_knr
# data_knr['lower range'] = y_test_pred_knr - (error_knr * y_test_pred_knr)
# data_knr['upper range'] = y_test_pred_knr + (error_knr * y_test_pred_knr)
# t_list = []
# for i in data_knr.index:
#     if data_knr['actual'][i] > data_knr['lower range'][i]:
#         if data_knr['actual'][i] < data_knr['upper range'][i]:
#             t_list.append('True')
#         else:
#             t_list.append('False')
#     else:
#         t_list.append('False')
# data_knr['Bool'] = t_list
# data_knr['Bool'].value_counts()/data_knr.shape[0]

In [ ]:
# svr = SVR()
# possible_parameter_values = {'gamma' : [float(x)/10000 for x in range(100001)],
#                             'C' : [float(x)/10 for x in range(1001)]}
# svr_rscv = RandomizedSearchCV(estimator = svr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
#                              n_jobs = 1)
# svr_rscv.fit(X_train, y_train)
# svr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [47]:
#y_train_pred_svr = svr_rscv.predict(X_train)
#y_test_pred_svr = svr_rscv.predict(X_test)
#print('Support Vector Regressor:')
#print('-' * 100)
#print('Train r2_score: ', r2_score(y_train, y_train_pred_svr))
#print('Test r2_score: ', r2_score(y_test, y_test_pred_svr))
#print('-' * 100)
#print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_svr)))
#print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_svr)))
#print('-' * 100)
#print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_svr) / y_train))
#print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_svr) / y_test))
#error_svr = np.mean(mean_absolute_error(y_test, y_test_pred_svr) / y_test)

In [48]:
#data_svr = pd.DataFrame()
#data_svr['actual'] = y_test
#data_svr['predicted'] = y_test_pred_svr
#data_svr['lower range'] = y_test_pred_svr - (error_svr * y_test_pred_svr)
#data_svr['upper range'] = y_test_pred_svr + (error_svr * y_test_pred_svr)
#t_list = []
#for i in data_svr.index:
#    if data_svr['actual'][i] > data_svr['lower range'][i]:
#        if data_svr['actual'][i] < data_svr['upper range'][i]:
#            t_list.append('True')
#        else:
#            t_list.append('False')
#    else:
#        t_list.append('False')
#data_svr['Bool'] = t_list
#data_svr['Bool'].value_counts()/data_svr.shape[0]

In [32]:
# xgbr = XGBRegressor()
# possible_parameter_values = {'n_estimators' : [int(x) for x in np.arange(start = 50, stop = 1001, step = 50)], 
#                              'use_label_encoder' : [True, False], 
#                              'max_depth' : [int(x) for x in np.arange(start = 5, stop = 101, step = 5)], 
#                              'learning_rate' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
# xgbr_rscv = RandomizedSearchCV(estimator = xgbr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, 
#                                random_state = 17, n_jobs = 1)
# xgbr_rscv.fit(X_train, y_train)
# xgbr_rscv.best_params_

In [33]:
# y_train_pred_xgbr = xgbr_rscv.predict(X_train)
# y_test_pred_xgbr = xgbr_rscv.predict(X_test)
# print('XGBoost Regressor:')
# print('-' * 100)
# print('Train r2_score: ', r2_score(y_train, y_train_pred_xgbr))
# print('Test r2_score: ', r2_score(y_test, y_test_pred_xgbr))
# print('-' * 100)
# print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_xgbr)))
# print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_xgbr)))
# print('-' * 100)
# print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_xgbr) / y_train))
# print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_xgbr) / y_test))
# error_xgbr = np.mean(mean_absolute_error(y_test, y_test_pred_xgbr) / y_test)

In [34]:
# data_xgbr = pd.DataFrame()
# data_xgbr['actual'] = y_test
# data_xgbr['predicted'] = y_test_pred_xgbr
# data_xgbr['lower range'] = y_test_pred_xgbr - (error_xgbr * y_test_pred_xgbr)
# data_xgbr['upper range'] = y_test_pred_xgbr + (error_xgbr * y_test_pred_xgbr)
# t_list = []
# for i in data_xgbr.index:
#     if data_xgbr['actual'][i] > data_xgbr['lower range'][i]:
#         if data_xgbr['actual'][i] < data_xgbr['upper range'][i]:
#             t_list.append('True')
#         else:
#             t_list.append('False')
#     else:
#         t_list.append('False')
# data_xgbr['Bool'] = t_list
# data_xgbr['Bool'].value_counts()/data_xgbr.shape[0]

In [53]:
file = open(r'D:\\IPL\\ipl_score_prediction.pkl', 'wb')
pickle.dump(abr_rscv, file)
file.close()

In [59]:
'batting_team_Chennai_Super_Kings', 'batting_team_Delhi_Capitals',
'batting_team_Gujarat_Lions', 'batting_team_Kings_XI_Punjab',
'batting_team_Kolkata_Knight_Riders', 'batting_team_Mumbai_Indians',
'batting_team_Rajasthan_Royals', 'batting_team_Rising_Pune_Supergiant',
'batting_team_Royal_Challengers_Bangalore',
'batting_team_Sunrisers_Hyderabad', 'bowling_team_Chennai_Super_Kings',
'bowling_team_Delhi_Capitals', 'bowling_team_Gujarat_Lions',
'bowling_team_Kings_XI_Punjab', 'bowling_team_Kolkata_Knight_Riders',
'bowling_team_Mumbai_Indians', 'bowling_team_Rajasthan_Royals',
'bowling_team_Rising_Pune_Supergiant',
'bowling_team_Royal_Challengers_Bangalore',
'bowling_team_Sunrisers_Hyderabad', 'toss_decision_bat',
'toss_decision_field',
'venue_Dr_YS_Rajasekhara_Reddy_ACA_VDCA_Cricket_Stadium',
'venue_Dubai_International_Cricket_Stadium', 'venue_Eden_Gardens',
'venue_Feroz_Shah_Kotla', 'venue_Green_Park',
'venue_Holkar_Cricket_Stadium', 'venue_M_Chinnaswamy_Stadium',
'venue_MA_Chidambaram_Stadium_Chepauk',
'venue_Maharashtra_Cricket_Association_Stadium',
'venue_Punjab_Cricket_Association_IS_Bindra_Stadium_Mohali',
'venue_Rajiv_Gandhi_International_Stadium_Uppal',
'venue_Saurashtra_Cricket_Association_Stadium',
'venue_Sawai_Mansingh_Stadium',
'venue_Shaheed_Veer_Narayan_Singh_International_Stadium',
'venue_Sharjah_Cricket_Stadium', 'venue_Sheikh_Zayed_Stadium',
'venue_Wankhede_Stadium', 'inning', 'over', 'ball', 'runs', 'wickets',
'runs_last_5_overs', 'wickets_last_5_overs'

('runs_last_5_overs', 'wickets_last_5_overs')

In [60]:
arr = np.array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 5, 0, 30, 1, 30, 1])

In [61]:
arr = arr.reshape(1, -1)

In [62]:
s = knr_rscv.predict(arr)[0]
s

165.5

In [63]:
s - (0.09185259348772307 * s)

150.29839577778182

In [64]:
s + (0.09185259348772307 * s)

180.70160422221818

In [31]:
int(5.3/1)

5

In [40]:
np.round((5.3%1)*10, 0)

3.0

In [35]:
np.round(7.4%1, 1)

0.4